Label\
000000015.jpg\
(37.53, 84.11, 81.98, 128.56)\
(5.33, 100.77, 49.78, 145.21)\
(41.97, 181.82, 86.42, 224.0)\
(99.71, 154.06, 144.16, 198.51)



Eric\
Psicina numero: 1, Coordenadas --> Xmin: 98, Ymin: 152, Xmax: 136, Ymax: 211\
Psicina numero: 2, Coordenadas --> Xmin: 9, Ymin: 105, Xmax: 39, Ymax: 140\
Psicina numero: 3, Coordenadas --> Xmin: 39, Ymin: 87, Xmax: 76, Ymax: 114

In [23]:
coordenadas_labels = [
    (37.53, 84.11, 81.98, 128.56),
    (5.33, 100.77, 49.78, 145.21),
    (41.97, 181.82, 86.42, 224.0),
    (99.71, 154.06, 144.16, 198.51)
]


coordenadas_labels_eric = [
    (98, 152, 136, 211),  # Piscina 1
    (9, 105, 39, 140),    # Piscina 2
    (39, 87, 76, 114),     # Piscina 3
]

def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

accuracys = []
for coordenada in coordenadas_labels:
	
	max_iou = 0
	for coordenada_eric in coordenadas_labels_eric:
		iou = bb_intersection_over_union(coordenada,coordenada_eric)
		if iou > max_iou:
			max_iou = iou
			#print(max_iou)
	accuracys.append(max_iou)
	
print(accuracys)

print(sum(accuracys)/len(accuracys))



[0.5150790106513401, 0.5403709385023008, 0, 0.6251975379139997]
0.4201618717669101


In [2]:
import load_clasification

images_with_pools, images_without_pools = load_clasification.clasification()

Vamos a juntar en un solo diccionario todo

In [4]:
all_filenames = sorted(list(images_with_pools.keys()) + sorted(images_without_pools))

oficial_labels = {}

for filename in all_filenames:
    if filename in images_with_pools:
        oficial_labels[filename] = images_with_pools[filename]
    else:
        oficial_labels[filename] = None  

In [1]:
import load_hsvANDcanny

hsv_and_canny_labels = load_hsvANDcanny.hsvCanny()

000000000.jpg: []
000000001.jpg: []
000000002.jpg: []
000000003.jpg: []
000000004.jpg: []
000000005.jpg: []
000000006.jpg: []
000000007.jpg: []
000000008.jpg: []
000000009.jpg: []
000000010.jpg: []
000000011.jpg: []
000000012.jpg: []
000000013.jpg: []
000000014.jpg: [(121, 105, 151, 140), (151, 87, 188, 114)]
000000015.jpg: [(98, 152, 136, 211), (9, 105, 39, 140), (39, 87, 76, 114)]
000000016.jpg: []
000000017.jpg: [(168, 94, 203, 139)]
000000018.jpg: []
000000019.jpg: []
000000020.jpg: [(56, 94, 91, 139)]
000000021.jpg: []
000000022.jpg: [(151, 140, 187, 191), (133, 74, 158, 120)]
000000023.jpg: [(39, 140, 75, 191), (21, 74, 46, 120)]
000000024.jpg: []
000000025.jpg: []
000000026.jpg: []
000000027.jpg: []
000000028.jpg: []
000000029.jpg: [(106, 138, 131, 182)]


In [5]:
print(oficial_labels)
print(hsv_and_canny_labels)



{'000000000.jpg': None, '000000001.jpg': None, '000000002.jpg': None, '000000003.jpg': None, '000000004.jpg': None, '000000005.jpg': None, '000000006.jpg': None, '000000007.jpg': None, '000000008.jpg': None, '000000009.jpg': None, '000000010.jpg': None, '000000011.jpg': None, '000000012.jpg': [(149.53, 196.11, 193.97, 224.0), (120.24, 212.77, 158.87, 224.0)], '000000013.jpg': [(37.53, 196.11, 81.97, 224.0), (8.24, 212.77, 46.87, 224.0)], '000000014.jpg': [(149.53, 84.11, 193.98, 128.56), (117.33, 100.77, 161.78, 145.21), (153.97, 181.82, 198.42, 224.0), (211.71, 156.41, 224.0, 196.16)], '000000015.jpg': [(37.53, 84.11, 81.98, 128.56), (5.33, 100.77, 49.78, 145.21), (41.97, 181.82, 86.42, 224.0), (99.71, 154.06, 144.16, 198.51)], '000000016.jpg': None, '000000017.jpg': [(167.73, 87.3, 212.18, 131.74), (122.21, 0.0, 166.65, 39.58)], '000000018.jpg': [(0.0, 154.06, 32.15, 198.51)], '000000019.jpg': [(167.88, 0.0, 212.04, 19.74)], '000000020.jpg': [(206.36, 218.32, 224.0, 224.0), (125.69, 

In [8]:
#Inicializamos el diccionario de accuracys
accuracy_hsv = {}

for item in oficial_labels.keys():
    accuracy_hsv[item] = None

In [13]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou



for item, labels in oficial_labels.items(): #iteramos sobre todo el diccionario de piscinas, por item y labels
    if labels != None:
        accuracys = []
        for hsv_label in hsv_and_canny_labels[item]: #miramos los labels encontrados en la piscina en concreto de hsv
            max_iou = 0
            for label in labels:
                iou = bb_intersection_over_union(label,hsv_label)
                if iou > max_iou:
                    max_iou = iou
                    #print(max_iou)
            accuracys.append(max_iou)
        #calculamos el promedio de accuracy en cada iteración
        accuracy_hsv[item] = sum(accuracys)/(len(accuracys) + 1)
    else:
        if (len(hsv_and_canny_labels[item]) == 0):
            accuracy_hsv[item] = "ACC: 100 - No hay piscina"
        else:
            accuracy_hsv[item] = "ACC: 0 - Se Detectó piscina sin haberla"

           


        

KeyError: '000000030.jpg'